In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
2023-01-26 18:17:18.590401: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Структура отчета по второму Заданию:

#### 1.       Постановка задачи

#### 2.       Реализация задачи классификации с использованием feedforward сети.

#### 3.       Реализация задачи с использованием LSTM слоев

#### 4.       Анализ

####          a.       Сравнение двух решений

####          b.       Поведение обученной нейронной сети на новых данных (предсказание на данных не входящих в train data set)

####          c.       Метрики, можно ли их улучшить?

####          d.       Ваши личные выводы из практического задания

### Report structure for the second Task:

#### 1. Statement of the problem

#### 2. Implementation of the classification problem using the feedforward network.

#### 3. Implementation of the task using LSTM layers

#### 4. Analysis

####    a. Comparison of two solutions

####    b. The behavior of the trained neural network on new data (prediction on data not included in the train data set)

####    c. Metrics, can they be improved?

####    d. Your personal conclusions from the practical task

In [2]:
dataset = pd.read_csv('actionDataset.csv',sep=';')

In [3]:
dataset

,id,Nose_x0,Nose_y0,Neck_x1,Neck_y1,RShoulder_x2,RShoulder_y2,Relbow_x3,RElbow_y3,RWrist_x4,...,REye_x14,REye_y14,LEye_x15,LEye_y15,REar_x16,REar_y16,LEar_x17,LEar_y17,action,video_src
0,1,76,23,82,33,80,32,73,38,61,...,76,21,78,22,0,0,83,23,boxing,person01_boxing_d1_uncomp.avi
1,2,76,22,82,32,80,31,71,36,62,...,76,20,78,20,0,0,82,22,boxing,person01_boxing_d1_uncomp.avi
2,3,77,21,83,33,81,31,72,36,63,...,0,0,79,20,0,0,82,22,boxing,person01_boxing_d1_uncomp.avi
3,4,78,21,84,32,82,31,73,37,64,...,79,20,80,20,0,0,84,22,boxing,person01_boxing_d1_uncomp.avi
4,5,78,21,84,31,82,31,77,41,67,...,0,0,79,20,0,0,84,22,boxing,person01_boxing_d1_uncomp.avi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179341,179342,18,19,26,23,22,23,0,0,0,...,0,0,19,16,0,0,22,16,walking,person25_walking_d4_uncomp.avi
179342,179343,16,16,23,23,0,0,0,0,0,...,0,0,17,14,0,0,20,14,walking,person25_walking_d4_uncomp.avi
179343,179344,0,0,21,22,19,22,0,0,0,...,0,0,15,15,0,0,16,14,walking,person25_walking_d4_uncomp.avi
179344,179345,0,0,9,22,0,0,0,0,0,...,0,0,8,14,0,0,11,15,walking,person25_walking_d4_uncomp.avi


In [4]:
dataset.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179346 entries, 0 to 179345
Data columns (total 39 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            179346 non-null  int64 
 1   Nose_x0       179346 non-null  int64 
 2   Nose_y0       179346 non-null  int64 
 3   Neck_x1       179346 non-null  int64 
 4   Neck_y1       179346 non-null  int64 
 5   RShoulder_x2  179346 non-null  int64 
 6   RShoulder_y2  179346 non-null  int64 
 7   Relbow_x3     179346 non-null  int64 
 8   RElbow_y3     179346 non-null  int64 
 9   RWrist_x4     179346 non-null  int64 
 10  RWrist_y4     179346 non-null  int64 
 11  LShoulder_x5  179346 non-null  int64 
 12  LShoulder_y5  179346 non-null  int64 
 13  LElbow_x6     179346 non-null  int64 
 14  LElbow_y6     179346 non-null  int64 
 15  LWrist_x7     179346 non-null  int64 
 16  LWrist_y7     179346 non-null  int64 
 17  RHip_x8       179346 non-null  int64 
 18  RHip_y8       179346 non

In [5]:
dataset = dataset.drop(columns=['id', 'video_src'])
dataset

,Nose_x0,Nose_y0,Neck_x1,Neck_y1,RShoulder_x2,RShoulder_y2,Relbow_x3,RElbow_y3,RWrist_x4,RWrist_y4,...,LAnkle_y13,REye_x14,REye_y14,LEye_x15,LEye_y15,REar_x16,REar_y16,LEar_x17,LEar_y17,action
0,76,23,82,33,80,32,73,38,61,36,...,103,76,21,78,22,0,0,83,23,boxing
1,76,22,82,32,80,31,71,36,62,36,...,102,76,20,78,20,0,0,82,22,boxing
2,77,21,83,33,81,31,72,36,63,35,...,0,0,0,79,20,0,0,82,22,boxing
3,78,21,84,32,82,31,73,37,64,36,...,100,79,20,80,20,0,0,84,22,boxing
4,78,21,84,31,82,31,77,41,67,37,...,0,0,0,79,20,0,0,84,22,boxing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179341,18,19,26,23,22,23,0,0,0,0,...,92,0,0,19,16,0,0,22,16,walking
179342,16,16,23,23,0,0,0,0,0,0,...,0,0,0,17,14,0,0,20,14,walking
179343,0,0,21,22,19,22,0,0,0,0,...,0,0,0,15,15,0,0,16,14,walking
179344,0,0,9,22,0,0,0,0,0,0,...,93,0,0,8,14,0,0,11,15,walking


In [6]:
dataset.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179346 entries, 0 to 179345
Data columns (total 37 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Nose_x0       179346 non-null  int64 
 1   Nose_y0       179346 non-null  int64 
 2   Neck_x1       179346 non-null  int64 
 3   Neck_y1       179346 non-null  int64 
 4   RShoulder_x2  179346 non-null  int64 
 5   RShoulder_y2  179346 non-null  int64 
 6   Relbow_x3     179346 non-null  int64 
 7   RElbow_y3     179346 non-null  int64 
 8   RWrist_x4     179346 non-null  int64 
 9   RWrist_y4     179346 non-null  int64 
 10  LShoulder_x5  179346 non-null  int64 
 11  LShoulder_y5  179346 non-null  int64 
 12  LElbow_x6     179346 non-null  int64 
 13  LElbow_y6     179346 non-null  int64 
 14  LWrist_x7     179346 non-null  int64 
 15  LWrist_y7     179346 non-null  int64 
 16  RHip_x8       179346 non-null  int64 
 17  RHip_y8       179346 non-null  int64 
 18  RKnee_x9      179346 non

In [29]:
dataset["action"]

0         0
1         0
2         0
3         0
4         0
         ..
179341    4
179342    4
179343    4
179344    4
179345    4
Name: action, Length: 179346, dtype: int64

In [9]:
X = dataset.drop(columns=['action'])
y = dataset[['action']]

In [10]:
X.shape

(179346, 36)

In [11]:
y.shape

(179346, 1)

In [12]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, random_state=42
)

In [13]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [14]:
X_train_scaled.shape

(143476, 36)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from keras.optimizers import SGD, Adam, Adadelta, RMSprop

In [16]:
model = Sequential()
model.add(Dense(60, input_dim= 36, activation = "sigmoid"))
model.add(Dense(15, activation = "sigmoid"))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "softmax"))
model.compile(Adam(lr = 0.01), "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 60)                2220      
                                                                 
 dense_1 (Dense)             (None, 15)                915       
                                                                 
 dropout (Dropout)           (None, 15)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 16        
                                                                 
Total params: 3,151
Trainable params: 3,151
Non-trainable params: 0
_________________________________________________________________


2023-01-26 18:17:30.997266: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/rsbek/opt/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [17]:
model.fit(X_train_scaled, y_train, verbose=1, epochs=10)

Epoch 1/10
4484/4484 [==============================] - 7s 1ms/step - loss: 0.0000e+00 - accuracy: 0.2360
Epoch 2/10
4484/4484 [==============================] - 6s 1ms/step - loss: 0.0000e+00 - accuracy: 0.2360
Epoch 3/10
4484/4484 [==============================] - 7s 1ms/step - loss: 0.0000e+00 - accuracy: 0.2360
Epoch 4/10
4484/4484 [==============================] - 7s 1ms/step - loss: 0.0000e+00 - accuracy: 0.2360
Epoch 5/10
4484/4484 [==============================] - 6s 1ms/step - loss: 0.0000e+00 - accuracy: 0.2360
Epoch 6/10
4484/4484 [==============================] - 6s 1ms/step - loss: 0.0000e+00 - accuracy: 0.2360
Epoch 7/10
4484/4484 [==============================] - 6s 1ms/step - loss: 0.0000e+00 - accuracy: 0.2360
Epoch 8/10
4484/4484 [==============================] - 6s 1ms/step - loss: 0.0000e+00 - accuracy: 0.2360
Epoch 9/10
4484/4484 [==============================] - 6s 1ms/step - loss: 0.0000e+00 - accuracy: 0.2360
Epoch 10/10
4484/4484 [=======================

In [18]:

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='tanh'),
    tf.keras.layers.Dense(256, activation='tanh'),
    tf.keras.layers.Dense(256, activation='tanh'),
    tf.keras.layers.Dense(1, activation='relu')
])

model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(lr=0.03),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

In [19]:
dataset1 = dataset.copy()
dataset1

,Nose_x0,Nose_y0,Neck_x1,Neck_y1,RShoulder_x2,RShoulder_y2,Relbow_x3,RElbow_y3,RWrist_x4,RWrist_y4,...,LAnkle_y13,REye_x14,REye_y14,LEye_x15,LEye_y15,REar_x16,REar_y16,LEar_x17,LEar_y17,action
0,76,23,82,33,80,32,73,38,61,36,...,103,76,21,78,22,0,0,83,23,0
1,76,22,82,32,80,31,71,36,62,36,...,102,76,20,78,20,0,0,82,22,0
2,77,21,83,33,81,31,72,36,63,35,...,0,0,0,79,20,0,0,82,22,0
3,78,21,84,32,82,31,73,37,64,36,...,100,79,20,80,20,0,0,84,22,0
4,78,21,84,31,82,31,77,41,67,37,...,0,0,0,79,20,0,0,84,22,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179341,18,19,26,23,22,23,0,0,0,0,...,92,0,0,19,16,0,0,22,16,4
179342,16,16,23,23,0,0,0,0,0,0,...,0,0,0,17,14,0,0,20,14,4
179343,0,0,21,22,19,22,0,0,0,0,...,0,0,0,15,15,0,0,16,14,4
179344,0,0,9,22,0,0,0,0,0,0,...,93,0,0,8,14,0,0,11,15,4


In [20]:
X = dataset.drop(columns=['action'])
y = dataset[['action']]

In [21]:
y = pd.get_dummies(dataset['action'])
y

,0,1,2,3,4
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
179341,0,0,0,0,1
179342,0,0,0,0,1
179343,0,0,0,0,1
179344,0,0,0,0,1


In [22]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, random_state=42
)

In [23]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [24]:
model = Sequential()
model.add(Dense(60, input_dim= 36, activation = "sigmoid"))
model.add(Dense(15, activation = "sigmoid"))
model.add(Dropout(0.2))
model.add(Dense(5, activation = "softmax"))
model.compile(Adam(lr = 0.01), "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 60)                2220      
                                                                 
 dense_8 (Dense)             (None, 15)                915       
                                                                 
 dropout_1 (Dropout)         (None, 15)                0         
                                                                 
 dense_9 (Dense)             (None, 5)                 80        
                                                                 
Total params: 3,215
Trainable params: 3,215
Non-trainable params: 0
_________________________________________________________________


/Users/rsbek/opt/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [25]:
model.fit(X_train_scaled, y_train, verbose=1, epochs=10)

Epoch 1/10
4484/4484 [==============================] - 7s 1ms/step - loss: 0.7261 - accuracy: 0.6885
Epoch 2/10
4484/4484 [==============================] - 6s 1ms/step - loss: 0.5657 - accuracy: 0.7687
Epoch 3/10
4484/4484 [==============================] - 6s 1ms/step - loss: 0.5241 - accuracy: 0.7824
Epoch 4/10
4484/4484 [==============================] - 6s 1ms/step - loss: 0.5006 - accuracy: 0.7918
Epoch 5/10
4484/4484 [==============================] - 6s 1ms/step - loss: 0.4791 - accuracy: 0.7992
Epoch 6/10
4484/4484 [==============================] - 6s 1ms/step - loss: 0.4687 - accuracy: 0.8034
Epoch 7/10
4484/4484 [==============================] - 6s 1ms/step - loss: 0.4568 - accuracy: 0.8086
Epoch 8/10
4484/4484 [==============================] - 6s 1ms/step - loss: 0.4506 - accuracy: 0.8119
Epoch 9/10
4484/4484 [==============================] - 6s 1ms/step - loss: 0.4417 - accuracy: 0.8157
Epoch 10/10
4484/4484 [==============================] - 6s 1ms/step - loss: 0.432

In [26]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, input_dim= 36, activation='tanh'),
    tf.keras.layers.Dense(256, activation='tanh'),
    tf.keras.layers.Dense(256, activation='tanh'),
    tf.keras.layers.Dense(5, activation='relu')
])

model2.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(lr=0.03),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)


In [27]:
model2.fit(X_train_scaled, y_train, verbose=1, epochs=20)

Epoch 1/20
4484/4484 [==============================] - 12s 2ms/step - loss: 7.3553 - accuracy: 0.5153 - precision: 0.1697 - recall: 0.3656
Epoch 2/20
4484/4484 [==============================] - 11s 3ms/step - loss: 3.0850 - accuracy: 0.8000 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 3/20
4484/4484 [==============================] - 10s 2ms/step - loss: 3.0850 - accuracy: 0.8000 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 4/20
4484/4484 [==============================] - 91s 20ms/step - loss: 3.0850 - accuracy: 0.8000 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 5/20
4484/4484 [==============================] - 12s 3ms/step - loss: 3.0850 - accuracy: 0.8000 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 6/20
4484/4484 [==============================] - 11s 3ms/step - loss: 3.0850 - accuracy: 0.8000 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 7/20
4484/4484 [==============================] - 12s 3ms/step - loss: 3.0850 - accuracy: 0.8000 - precision: 0.0

In [30]:
dataset['action'].value_counts()

2    52015
0    44300
1    42358
4    28815
3    11858
Name: action, dtype: int64

In [33]:
df = dataset

In [34]:
X1 = df.drop(columns=['action'])
y1 = df[['action']]

In [35]:
X_train, X_test, y_train, y_test = train_test_split(
    X1, y1, 
    test_size=0.2, random_state=42
)

In [36]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [37]:
X_train_scaled1 = X_train_scaled.reshape(X_train_scaled.shape[0],1,X_train_scaled.shape[1])

In [38]:
X_test_scaled1 = X_test_scaled.reshape(X_test_scaled.shape[0],1,X_test_scaled.shape[1])

In [39]:
(X_train_scaled1.shape, X_test_scaled1.shape)

((143476, 1, 36), (35870, 1, 36))

In [40]:
from keras.utils import np_utils 
Y_train = np_utils.to_categorical(y_train)

In [41]:
Y_train.shape

(143476, 5)

In [42]:
from keras.layers import LSTM
output_dim = 5
input_dim = X_train_scaled.shape[1]

In [44]:
batch_size = 512
nb_epoch = 20
model_lstm =Sequential()
model_lstm.add(LSTM(100))
model_lstm.add(Dense(output_dim, input_dim=input_dim, activation='sigmoid'))

#model_lstm.summary()

model_lstm.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [56]:
model_lstm.fit(X_train_scaled1, Y_train, epochs=50,batch_size=512)

Epoch 1/50
281/281 [==============================] - 2s 7ms/step - loss: 0.1154 - accuracy: 0.8861
Epoch 2/50
281/281 [==============================] - 2s 7ms/step - loss: 0.1156 - accuracy: 0.8859
Epoch 3/50
281/281 [==============================] - 2s 7ms/step - loss: 0.1155 - accuracy: 0.8858
Epoch 4/50
281/281 [==============================] - 2s 7ms/step - loss: 0.1154 - accuracy: 0.8863
Epoch 5/50
281/281 [==============================] - 2s 7ms/step - loss: 0.1156 - accuracy: 0.8860
Epoch 6/50
281/281 [==============================] - 2s 7ms/step - loss: 0.1151 - accuracy: 0.8868
Epoch 7/50
281/281 [==============================] - 2s 7ms/step - loss: 0.1153 - accuracy: 0.8861
Epoch 8/50
281/281 [==============================] - 2s 7ms/step - loss: 0.1151 - accuracy: 0.8864
Epoch 9/50
281/281 [==============================] - 2s 7ms/step - loss: 0.1154 - accuracy: 0.8858
Epoch 10/50
281/281 [==============================] - 2s 7ms/step - loss: 0.1149 - accuracy: 0.8864